In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import math

D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1ty

In [2]:
RandomSeed = 9
np.random.seed(RandomSeed)
tf.set_random_seed(RandomSeed)

In [3]:
class PtPINNsss:
    # Initialize the class
    def __init__(self, x, t, u, lb, ub, lbp,ubp, layers,result_weights_values,result_biases_values):
        
        X = np.concatenate([x, t], 1)

        self.X = X
        
        self.x = X[:,0:1]
        self.t = X[:,1:2]
        
        self.u = u      
        self.lb = lb
        self.ub = ub
        self.hsadasjd=0
        self.ubp = ubp
        self.lbp = lbp        
        # Initialize NNs
        self.layers = layers
        self.hh=[]
        self.hh1=[]      
        
        self.weights, self.biases = self.initialize_NN(layers)
        
        # Calculate the values and convert them to float32
        value_float64 = 0.7
        self.b = tf.Variable(value_float64, dtype=tf.float64)
        
        value_float64=  1.0/(math.cosh(5*0.5)-1)       
        
        self.a = tf.constant(value_float64, dtype=tf.float64)     
        
        
        
        self.weights_values = result_weights_values        
        self.biases_values = result_biases_values    

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))    

        self.x_f_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.x_lb_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_b_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        
        self.x_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float64, shape=[None, self.u.shape[1]])

        self.u_pred, _ ,self.uuuuuu = self.net_AC(self.x_tf, self.t_tf)
        self.u_lb_pred, self.ux_lb_pred,_ = self.net_AC(self.x_lb_tf, self.t_b_tf)
        self.u_ub_pred, self.ux_ub_pred,_ = self.net_AC(self.x_ub_tf, self.t_b_tf)

        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        

        
        self.lossS = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))                                              
        self.lossB = tf.reduce_mean(tf.square(self.u_lb_pred - self.u_ub_pred)) + tf.reduce_mean(tf.square(self.ux_lb_pred - self.ux_ub_pred))
        self.lossfu =  tf.reduce_mean(tf.square(self.f_pred))    
        

        self.optimizer_Adam = tf.train.AdamOptimizer()

        self.loss  =   self.lossB + self.lossfu
        
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)      

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    
        init = tf.global_variables_initializer()
        self.sess.run(init)
        self.save = tf.train.Saver(max_to_keep=1)
              
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = tf.Variable(tf.zeros([layers[l], layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    
    def initialize_NN2(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases      
    
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, dtype=tf.float64))
    
    def custom_function(self,t):
        ab= self.b     
        condition2 = tf.less_equal(t, 0.6)
        condition3 = tf.less_equal(t, ab)
        condition4 = tf.less_equal(ab, 0.8) 
        BB=5*(t-0.6)
        B=1/(ab-0.6)*(t-0.6)
        
        return tf.where(condition2, tf.zeros_like(t), tf.where(condition4,tf.where(condition3, -2*B**3+3*B**2,tf.zeros_like(t)+1),-2*BB**3+3*BB**2))
    
 

    def input_encoding(self, t, x):    
        H=tf.concat([t,1+0*tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.sin(math.pi*x)],1) 
        H=tf.concat([H,tf.cos(2*math.pi*x)],1)
        H=tf.concat([H,tf.sin(2*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(3*math.pi*x)],1)
        H=tf.concat([H,tf.sin(3*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(4*math.pi*x)],1)
        H=tf.concat([H,tf.sin(4*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(5*math.pi*x)],1)
        H=tf.concat([H,tf.sin(5*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(6*math.pi*x)],1)
        H=tf.concat([H,tf.sin(6*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(7*math.pi*x)],1)
        H=tf.concat([H,tf.sin(7*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(8*math.pi*x)],1)
        H=tf.concat([H,tf.sin(8*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(9*math.pi*x)],1)
        H=tf.concat([H,tf.sin(9*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(10*math.pi*x)],1)
        H=tf.concat([H,tf.sin(10*math.pi*x)],1)           
        
        return H


    def neural_net(self, x,t, weights, biases,weights_values,biases_values):
        X = tf.concat([x,t],1)
        tt=5*t
        num_layers = len(weights) + 1
        H = self.input_encoding(tt, x)
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            W1 = weights_values[l]
            b1 = biases_values[l]   
            H0 = tf.add(tf.matmul(H, W), b)
            H1 = tf.add(tf.matmul(H, W1), b1)
            
            B = self.custom_function(t)   
            H0 = H0*B              
            
            H  = tf.add(H0,H1)
            H =  tf.tanh(H)
        
        W = weights[-1]
        b = biases[-1]
        W1 = weights_values[-1]
        b1 = biases_values[-1]  
        
        H0 = tf.add(tf.matmul(H, W), b)
        
        B = self.custom_function(t)   
        H0 = H0*B            
        
        H1 = tf.add(tf.matmul(H, W1), b1) 
        H  = tf.add(H0,H1)      
        Y = H
        return Y
    
    def net_AC(self, x, t):
        u = self.neural_net(x,t, self.weights, self.biases,self.weights_values, self.biases_values)
        u_x = tf.gradients(u, x)[0]
        u_t = tf.gradients(u, t)[0]

        return u, u_x, u_t
          
    

    def net_f(self, x, t):
        u, u_x, u_t = self.net_AC(x, t)
        
        u_xx = tf.gradients(u_x, x)[0]
        
        f_u = u_t - 0.0001*u_xx+5*u**3-5*u
        
        return f_u
    
    def callback(self, loss, a,b,lossfu, lossS, lossB):
        sss=self.hsadasjd
        if sss%1000==0:
            print('Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e ' % (loss, lossfu, lossS, lossB))
        sss=sss+1
        self.hsadasjd=sss 
        self.hh.append(a)      
        self.hh1.append(b)        
    def train(self, nIter, Nf, Nn, Nb):

        X_train = self.lbp + (self.ubp-self.lbp)*lhs(2, Nf)
        self.xtrain_f = X_train[:,0:1]
        self.ttrain_f = X_train[:,1:2] 
        
        X_lb_train = self.lbp + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_lb = X_lb_train[:,0:1]
        self.ttrain_b = X_lb_train[:,1:2]
        
        X_ub_train = [self.ubp[0],self.lbp[1]] + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_ub = X_ub_train[:,0:1]
        
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.u_tf: self.u,
                   self.x_lb_tf: self.xtrain_lb, self.t_b_tf: self.ttrain_b, 
                   self.x_ub_tf: self.xtrain_ub, 
                   self.x_f_tf: self.xtrain_f, self.t_f_tf: self.ttrain_f}

        for it in range(nIter):
            loss_value = self.sess.run(self.loss, tf_dict)
            lossfu = self.sess.run(self.lossfu, tf_dict)
            lossS = self.sess.run(self.lossS, tf_dict)
            lossB = self.sess.run(self.lossB, tf_dict)
            self.sess.run(self.train_op_Adam, tf_dict)
                
            ab= self.sess.run(self.a)
            self.hh.append(ab)       
            abc= self.sess.run(self.b)
            self.hh1.append(abc)    

            print(abc)                    
            if it % 1000 == 0:
                print('It: %d, Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e' % (it, loss_value, lossfu, lossS, lossB))
                        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, method = 'L-BFGS-B', options = {'maxiter': 50000,'maxfun': 50000,'maxcor': 50,'maxls': 50,'ftol' : 1.0 * np.finfo(float).eps})                                                                                                         
        self.optimizer.minimize(self.sess, feed_dict = tf_dict, fetches = [self.loss,self.a,self.b,  self.lossfu, self.lossS, self.lossB], loss_callback = self.callback)        
                                    
    
    def predict(self, x, t):
        
        tf_dict = {self.x_tf: x, self.t_tf: t}
        u_star = self.sess.run(self.u_pred, tf_dict)
        u_starssss = self.sess.run(self.uuuuuu, tf_dict)        
        return u_star,u_starssss

    def saver(self, string):
        self.save.save(self.sess, 'ckpt/'+string)
        
        
    def sssss(self):
        return self.hh,  self.hh1           
        
        
    def restore(self):
        model_file = tf.train.latest_checkpoint('ckpt/')
        self.save.restore(self.sess, model_file)

In [4]:
import pickle

In [5]:
with open('bcweights.pkl', 'rb') as f:
    weights_values = pickle.load(f)

In [6]:
with open('bcweights1.pkl', 'rb') as f:
    biases_values = pickle.load(f)

In [7]:
with open('2bcweights.pkl', 'rb') as f:
    weights_values2 = pickle.load(f)
with open('2bcweights1.pkl', 'rb') as f:
    biases_values2 = pickle.load(f)

In [8]:
with open('3bcweights.pkl', 'rb') as f:
    weights_values3 = pickle.load(f)
with open('3bcweights1.pkl', 'rb') as f:
    biases_values3 = pickle.load(f)

In [9]:
wwweights_values = [x + y for x, y in zip(weights_values, weights_values2)]
rrrresult_biases_values = [x + y for x, y in zip(biases_values, biases_values2)]

In [10]:
result_weights_values = [x + y for x, y in zip(wwweights_values, weights_values3)]
result_biases_values = [x + y for x, y in zip(rrrresult_biases_values, biases_values3)]

In [11]:
if __name__ == "__main__": 
           
    
    # Doman bounds
    lb = np.array([-1.0, 0])
    ub = np.array([1.0, 0.2])

    # Pre-training interval
    lbp = np.array([-1.0, 0.6])    
    ubp = np.array([1.0,  0.8])


    
    layers = [22, 100, 100, 100, 100, 1]
    
    data = scipy.io.loadmat('AC.mat')
    
    t = data['tt'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = data['uu']
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]

    def IC(x):
        u = x**2*np.cos(np.pi*x)
        return u

    N0 = 1200
    x=np.linspace(-1,1,N0).flatten()[:,None]  
    X0 =x
    T0 = np.full((N0,1), lb[1])
    U0 = IC(X0)
      
    model2 = PtPINNsss(X0, T0, U0, lb, ub,lbp, ubp, layers,result_weights_values,result_biases_values)                        
    model2.train(5000, 4000, 1200, 1200)  
    model2.saver('testmodel.ckpt')
    


    u_pred ,_= model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0.7
It: 0, Loss: 1.351447e-01, Lossfu: 1.351e-01, LossS: 5.708e-02, LossB: 3.022e-29
0.7007441343333327
0.7014168043562053
0.7020921175376099
0.7027619325284622
0.7033521969941002
0.7038987805966842
0.7044489417329699
0.7050086681532195
0.705552026403843
0.7060517285387422
0.7065071257113096
0.7069462974052239
0.7073860852860722
0.7078156107972573
0.7082196884583699
0.708594526661808
0.7089435104481198
0.7092704094070965
0.7095766930982695
0.7098629589808609
0.7101318394633979
0.710388569000774
0.7106373733948822
0.7108786266115751
0.7111097978066303
0.7113289432814711
0.7115350498199011
0.7117275416389747
0.7119054702537619
0.7120686420565551
0.7122190878961869
0.7123607433746774
0.7124970847631038
0.7126288442781673
0.7127545847953909
0.7128728534198644
0.7129834834960521
0.713087266749295
0.7131852234247595
0.713278209870034
0.713367215563441
0.7134533390687148
0.7

0.7203481421971052
0.7203713690424576
0.7203945863974979
0.7204178134579333
0.7204410463219703
0.7204643095288432
0.7204875971578675
0.720510903837695
0.7205342281860125
0.7205575654328759
0.7205809112501784
0.7206042617062608
0.7206276240350568
0.7206510075349781
0.7206744069676977
0.7206978341115304
0.7207212830135613
0.7207447483436747
0.7207682252027613
0.7207917187389379
0.7208152239420489
0.7208387362726785
0.7208622626113389
0.7208857981422244
0.7209093385015617
0.7209329148538216
0.7209565208105894
0.7209801499925355
0.7210037966213576
0.721027456485052
0.7210511247499888
0.7210747970331983
0.7210984866804666
0.7211221883540633
0.7211458972182481
0.7211696114889627
0.7211933269247243
0.7212170402217654
0.7212407478475196
0.7212644465998842
0.721288133607625
0.7213118063303774
0.7213354623719501
0.7213591062266996
0.721382735250079
0.7214063470451065
0.7214299451976957
0.7214535271503985
0.721477105473349
0.7215006765485781
0.721524262722428
0.7215478832105965
0.7215715305875562

0.7302386341566903
0.7302590276865135
0.7302794268128753
0.7302998265450755
0.7303202223940692
0.7303406104511161
0.7303609870790733
0.7303814015685767
0.7304018456985812
0.7304223120587762
0.7304427939674883
0.7304632861099748
0.7304837829963635
0.7305042801266606
0.7305247730123401
0.730545257616302
0.7305657312428371
0.7305862226905631
0.7306067257003205
0.7306272906219258
0.730647906762595
0.7306685644775405
0.7306893036348596
0.7307101115118484
0.73073099069408
0.7307519294293334
0.7307729171222991
0.7307939442209994
0.7308150023382778
0.7308360837015708
0.7308571813126312
0.7308782987909822
0.7308994829389247
0.7309207223746831
0.7309420068373496
0.7309633336760253
0.7309846949575171
0.7310060824748941
0.7310274888387359
0.7310489074014899
0.7310703321788952
0.7310917577850534
0.7311131798522347
0.7311345939772039
0.7311560159877848
0.7311774403640041
0.7311988645815641
0.7312202839225913
0.7312417141285301
0.7312631494051964
0.731284584548683
0.7313060148788992
0.73132743623534


0.7396477638757045
0.7396649902927426
0.7396821936528956
0.7396993734649753
0.7397165292931293
0.7397336611906256
0.7397507687678255
0.7397678516802566
0.7397849096261231
0.7398019423383356
0.7398189491214555
0.7398359298171636
0.7398529162848274
0.7398699051885363
0.739886893526663
0.739903878594784
0.7399209042071908
0.7399379635164344
0.7399550503491774
0.7399721591403828
0.7399892848739598
0.7400064230314475
0.7400235818175993
0.7400407563413538
0.7400579431860779
0.7400751382839024
0.7400923379695831
0.7401095455273436
0.7401267573352301
0.7401439711867037
0.7401611840440655
0.7401783931812213
0.7401955961382999
0.7402127907406763
0.7402299749959621
0.7402471471794663
0.74026430567849
0.7402814490647052
0.7402985784581693
0.7403156924744945
0.7403327898826384
0.7403498695769417
0.7403669305267639
0.7403839718623957
0.7404009955053793
0.7404180005052867
0.7404349899415045
0.7404519626773611
0.7404689175559306
0.7404858537164585
0.7405027701713308
0.7405196662481756
0.74053654106095

0.748322402866746
0.7483364347889333
0.7483504747611259
0.7483645204340428
0.7483785696942838
0.7483926206352964
0.7484066922777557
0.7484207809987757
0.748434886431174
0.7484490053405972
0.7484631348211429
0.7484772722440912
0.7484914152515111
0.7485055617062829
0.7485197113490555
0.7485338634258559
0.7485480161342603
0.7485621678235145
0.7485763170460349
0.7485904624524505
0.7486046028815707
0.7486187372260004
0.7486328645941597
0.7486469840638847
0.7486610949334226
0.7486751964129386
0.7486892879950395
0.7487033689692247
0.7487174390772602
0.7487314976199756
0.7487455445772397
0.7487595791681424
0.748773601598775
0.7487876118004271
0.7488016103346895
0.7488155956747967
0.7488295691344561
0.7488435301608519
0.7488574811628117
0.7488714180138246
0.7488853454326981
0.7488992563406793
0.7489131596573225
0.7489270438066468
0.7489409254036025
0.748954781113546
0.7489686423826327
0.7489824679979411
0.7489963176725808
0.7490101085543796
0.7490239563301455
0.7490377094671059
0.74905159430131

...

0.7797004999869385
0.7797072983860949
0.7797140631495377
0.7797208398331238
0.7797276203036341
0.7797343724997303
0.7797411498960126
0.7797479158981138
0.7797546785398558
0.779761463666866
0.7797682307468891
0.7797750120566329
0.7797818031834886
0.7797885894611888
0.7797953949547127
0.7798022004023465
0.779809013745084
0.7798158409507123
0.7798226689664389
0.779829507417146
0.779836353316799
0.7798432063258719
0.779850065691623
0.7798569321081699
0.779863806131069
0.7798706819454643
0.7798775671198187
0.7798844544523714
0.7798913465487292
0.7798982502225699
0.7799051537329323
0.7799120669450352
0.779918985226882
0.779925903837483
0.7799328336828103
0.7799397640569438
0.7799467002705288
0.7799536436130289
0.7799605861175734
0.7799675368493301
0.7799744904643395
0.779981447345104
0.7799884114781097
0.7799953768910259
0.7800023475484731
0.7800093225207755
0.7800163008323445
0.7800232845008931
0.7800302720536946
0.7800372641074039
0.7800442599264034
0.780051260489316
0.780058264650129
0.78

0.7885864986881342
0.7885920266894143
0.7885975735949271
0.788603110517943
0.7886086319418169
0.7886141711151999
0.7886196921287904
0.7886252136250902
0.7886307456583329
0.7886362590347215
0.7886417830143019
0.7886473073403028
0.7886528208008061
0.7886583502502863
0.7886638739276284
0.7886693976752176
0.7886749336309969
0.7886804616677591
0.7886859991952395
0.7886915436200852
0.7886970848143144
0.7887026392958765
0.7887081940171857
0.7887137502326138
0.788719317795389
0.7887248835215355
0.7887304569255579
0.7887360365722638
0.7887416141952593
0.7887472010548567
0.788752788732186
0.7887583777566894
0.7887639744292095
0.788769568989339
0.788775167685614
0.7887807700020871
0.7887863719760679
0.7887919810395472
0.7887975914391349
0.788803202998476
0.7888088199462804
0.7888144357028337
0.7888200546818462
0.7888256765108583
0.7888312973849626
0.7888369220788862
0.7888425465017025
0.7888481711867417
0.7888537988541664
0.7888594254368414
0.7888650539076643
0.7888706835478028
0.7888763123470675

0.7977736097944685
0.7978147381855307
0.7978532160926413
0.7978890820882497
0.7979224146033511
0.7979541138886829
0.7979831383216722
0.79801056187111
0.7980361329389849
0.7980597315750364
0.7980818396416205
0.7981025177319921
0.798122147446016
0.798140215063889
0.7981573129899966
0.7981734262917943
0.7981881594324917
0.7982021451692941
0.7982151436492543
0.7982272367536976
0.7982389488092783
0.7982498472799017
0.7982600597720372
0.7982698857104784
0.7982789806447752
0.7982876628026326
0.7982959462863697
0.7983037182988953
0.7983111887499148
0.7983183004552817
0.7983251685422277
0.7983316408300036
0.798337879749235
0.798343878809263
0.7983495063685715
0.7983550280535012
0.798360316656669
0.7983653845460242
0.7983703839316727
0.7983751767279349
0.7983798035014854
0.7983843787803024
0.7983887455463609
0.798393020470214
0.798397216288815
0.7984012768147218
0.7984052835174578
0.7984092043935427
0.7984130863687288
0.7984168761028163
0.7984206372649313
0.7984243486930687
0.798427991462409
0.7

0.8015360302950292
0.8015360302950293
0.8015360302950294
0.8015360302950295
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302

0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302950296
0.8015360302

In [24]:
    t = data['tt'].flatten()[:,None][120:161] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,120:161] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.80)
Error2 u: 5.491315e-04
Error1 u: 2.295022e-04
Errorf u: 3.001049e-03


In [25]:
    scipy.io.savemat("solution4.mat", {'u': u_pred})

In [26]:
    t = data['tt'].flatten()[:,None][121:161] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,121:161] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,0.80)
Error2 u: 5.509384e-04
Error1 u: 2.301138e-04
Errorf u: 3.001049e-03


In [27]:
    scipy.io.savemat("solution44.mat", {'u': u_pred})

In [22]:
weights_values2 = model2.sess.run(model2.weights)
biases_values2 = model2.sess.run(model2.biases)

In [23]:
import pickle
with open('4bcweights.pkl', 'wb') as f:
    pickle.dump(weights_values2, f)
with open('4bcweights1.pkl', 'wb') as f:
    pickle.dump(biases_values2, f)